In [ ]:
import sys
import os

# 現在: mtsb/notebooks
# 1つ上は: mtsb/
PROJECT_ROOT = os.path.abspath("..")
sys.path.append(PROJECT_ROOT)

print("Added to sys.path:", PROJECT_ROOT)


Added to sys.path: c:\Users\AVRJ0006\YamatoDX\YAMATO_learning_practice\my_time_series_benchmark\project_git\My-Time-Series-Benchmark\mtsb


In [ ]:
# データ読み込み
import pandas as pd

df = pd.read_csv("../data/raw/daily_sales.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

y = df['sales']

# 30日先予測（ベースモデルは未来特徴量を使わないためこれでOK）
train = y.iloc[:-30]
test = y.iloc[-30:]

In [ ]:
# ナイーブモデルを適用
from models.baseline.naive import NaiveModel

horizon = 30
model = NaiveModel(freq="D")
model.fit(train)
y_pred = model.y_pred(horizon)

In [8]:
from evaluation.basic.mae import MAE
from evaluation.basic.rmse import RMSE
from evaluation.basic.mape import MAPE
from evaluation.basic.smape import SMAPE
from evaluation.basic.r2 import R2

metrics = {
    "mae": MAE().compute(test, y_pred),
    "rmse": RMSE().compute(test, y_pred),
    "mape": MAPE().compute(test, y_pred),
    "smape": SMAPE().compute(test, y_pred),
    "r2": R2().compute(test, y_pred),
}

print(metrics)


{'mae': 13.699333333333337, 'rmse': 17.17573637431595, 'mape': 9.182236809347451, 'smape': 9.850489519326375, 'r2': -1.2351626365649384}


In [12]:
# 予測結果をCSVで保存
y_pred_df = pd.DataFrame({
    "date": y_pred.index,
    "actual": test.values,
    "y_pred": y_pred.values,
    "model": "naive"
})

y_pred_df.to_csv("../../exports_for_tableau/naive_y_pred.csv", index=False)


In [13]:
# 評価指標をCSVで保存
metrics_df = pd.DataFrame([{
    "model": "naive",
    **metrics
}])

metrics_df.to_csv("../../exports_for_tableau/naive_metrics.csv", index=False)


In [14]:
import numpy as np
import pandas as pd

errors = []
for h in range(1, 31):
    errors.append({
        "horizon": h,
        "mae": abs(test.iloc[h-1] - y_pred.iloc[h-1]),
        "rmse": (test.iloc[h-1] - y_pred.iloc[h-1]) ** 2,
        "mape": abs(test.iloc[h-1] - y_pred.iloc[h-1]) / abs(test.iloc[h-1]) * 100
    })

horizon_df = pd.DataFrame(errors)
horizon_df.to_csv("../../exports_for_tableau/naive_horizon_error.csv", index=False)


In [15]:
residual = test - y_pred

residual_df = pd.DataFrame({
    "residual": residual,
    "acf_1": residual.autocorr(lag=1),
    "acf_7": residual.autocorr(lag=7),
})
residual_df.to_csv("../../exports_for_tableau/naive_residual.csv")
